In [32]:
import pandas as pd
import numpy as np
import spacy
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from datetime import date
import pickle

In [33]:
df=pd.read_csv(r"C:\Users\vysak\OneDrive\Desktop\python\ai_chatbot_data_new")
df=df.iloc[:,1:]
df

,intent,queries
0,greetings,hello
1,greetings,hi
2,greetings,hello is there anyone
3,greetings,hola
4,greetings,hey
5,greetings,good morning
6,greetings,good evening
7,greetings,whats up
8,name,What is your name
9,name,Can i know who you are


In [34]:
nlp=spacy.load("en_core_web_sm")

In [35]:
vectors=[]
for i in df['queries']:
    doc=nlp(i)
    vectors.append(doc.vector)

In [36]:
df['vector_query']=vectors
df

,intent,queries,vector_query
0,greetings,hello,"[-0.26057333, -0.43720353, 0.19339824, 0.23490..."
1,greetings,hi,"[-0.13233288, -1.2025864, 0.56847215, 0.408427..."
2,greetings,hello is there anyone,"[0.0455539, -0.23895001, -0.3251374, 0.1008425..."
3,greetings,hola,"[-0.63406897, -0.4778974, 0.0018317103, 0.5175..."
4,greetings,hey,"[-0.67481184, -0.8644798, -0.29753524, 0.10506..."
5,greetings,good morning,"[1.1536707, -0.90012527, 0.464602, -0.78810054..."
6,greetings,good evening,"[1.1115007, -0.79802495, -0.025956634, -0.6767..."
7,greetings,whats up,"[0.31687695, 0.06751486, 0.3640482, 0.18154196..."
8,name,What is your name,"[0.12949863, -0.065115735, 0.5040627, 0.148717..."
9,name,Can i know who you are,"[-0.2473812, -0.36464444, 0.11002389, -0.13895..."


In [37]:
df['intent_num']=df.intent.map({'greetings':0,'name':1,'date':2,'closing':3,'wellness':4,'random':5,'owner':6})
df

,intent,queries,vector_query,intent_num
0,greetings,hello,"[-0.26057333, -0.43720353, 0.19339824, 0.23490...",0
1,greetings,hi,"[-0.13233288, -1.2025864, 0.56847215, 0.408427...",0
2,greetings,hello is there anyone,"[0.0455539, -0.23895001, -0.3251374, 0.1008425...",0
3,greetings,hola,"[-0.63406897, -0.4778974, 0.0018317103, 0.5175...",0
4,greetings,hey,"[-0.67481184, -0.8644798, -0.29753524, 0.10506...",0
5,greetings,good morning,"[1.1536707, -0.90012527, 0.464602, -0.78810054...",0
6,greetings,good evening,"[1.1115007, -0.79802495, -0.025956634, -0.6767...",0
7,greetings,whats up,"[0.31687695, 0.06751486, 0.3640482, 0.18154196...",0
8,name,What is your name,"[0.12949863, -0.065115735, 0.5040627, 0.148717...",1
9,name,Can i know who you are,"[-0.2473812, -0.36464444, 0.11002389, -0.13895...",1


In [7]:
x=df['vector_query']
y=df['intent_num']

In [8]:
x=np.array(x)
x

array([array([-0.26057333, -0.43720353,  0.19339824,  0.23490821, -0.21584016,
              -0.43348736, -0.5151609 ,  0.16586182, -0.08581167,  0.40069428,
              -0.5235219 , -0.7207984 ,  0.90150416, -0.5878126 , -0.97701025,
              -0.09032369, -0.220023  , -1.8936089 , -0.14577624,  0.2855091 ,
              -1.3107332 ,  0.82541484, -0.55664045, -0.12928464,  0.3367369 ,
               0.09734668,  0.6715669 , -0.20630707, -2.0879452 ,  0.73995525,
               0.6676406 ,  0.42990124, -0.1743044 ,  1.1478373 ,  0.15635663,
              -0.73395693, -0.5419658 , -0.00306262,  0.35015625,  1.0658401 ,
              -1.0770184 , -0.2040222 ,  0.63144183,  0.09003377, -0.14965132,
               0.13181193, -0.38989204,  0.0129724 ,  0.8450628 , -1.2045038 ,
               1.8880705 ,  0.71002316, -0.38820615,  0.1554792 , -0.05142748,
               0.9726086 ,  2.2668445 , -1.0222335 , -0.32422233, -0.6407145 ,
              -0.37645593, -0.24688962, -0.6910719 ,

In [9]:
x=np.stack(x)
x

array([[-0.26057333, -0.43720353,  0.19339824, ..., -0.2495521 ,
        -0.29881614,  0.96008515],
       [-0.13233288, -1.2025864 ,  0.56847215, ...,  0.55424595,
        -0.4794532 ,  0.5846106 ],
       [ 0.0455539 , -0.23895001, -0.3251374 , ..., -0.22629248,
        -0.60768497,  0.27560946],
       ...,
       [-0.54827344,  0.03370478, -0.2717325 , ..., -0.42345697,
        -0.12041388,  0.47611564],
       [-0.70588595, -0.09804446, -0.25055084, ..., -0.7958844 ,
         0.5576142 , -0.3888494 ],
       [ 0.16067544,  0.14451289, -0.03936896, ..., -0.7037852 ,
         0.02959032,  0.4636994 ]], dtype=float32)

In [10]:
y=np.array(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3,
       3, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 6, 6, 6, 6, 6, 6, 6, 6, 6], dtype=int64)

In [11]:
y=to_categorical(y)
y

array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
 

In [12]:
x.shape

(54, 96)

In [13]:
model=Sequential()
model.add(Dense(128,input_shape=(96,),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7,activation='softmax'))

sgd=SGD(learning_rate=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])


C:\Users\vysak\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\vysak\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [14]:
model.fit(x,y,epochs=200,batch_size=5,verbose=1)

Epoch 1/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2051 - loss: 2.1382  
Epoch 2/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2967 - loss: 1.8960     
Epoch 3/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2599 - loss: 1.7041 
Epoch 4/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5181 - loss: 1.3435 
Epoch 5/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4330 - loss: 1.2895 
Epoch 6/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5670 - loss: 1.3358 
Epoch 7/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6610 - loss: 1.0127 
Epoch 8/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7370 - loss: 1.0094 
Epoch 9/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6798 - loss: 0.8983 
Epoch 10/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7163 - loss: 0.9292 
Epoch 11/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7308 - loss: 0.8704 
Epoch 12/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9787 - loss: 0.0321 
Epoch 136/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9481 - loss: 0.1140 
Epoch 137/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9934 - loss: 0.0105     
Epoch 138/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9952 - loss: 0.0259     
Epoch 139/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0059 
Epoch 140/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0075 
Epoch 141/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9952 - loss: 0.0129     
Epoch 142/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0546     
Epoch 143/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9781 - loss: 0.0536     
Epoch 144/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0510 
Epoch 145/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0171     
Epoch 146/200
11/11 ━━

In [17]:
# SAVING MODEL

with open('helpai.pkl','wb') as f:
    pickle.dump(model,f)
    

In [38]:
# LOADING MODEL

with open('helpai.pkl','rb') as f:
    model=pickle.load(f)

In [39]:
def preprocess(txt):
    doc=nlp(txt)
    inp_dat=doc.vector
    reshaped_data = np.reshape(inp_dat, (1, 96))
    return reshaped_data

def pred(res):
    out_val=model.predict([res])
    return np.array(out_val)

In [40]:
import random
date_tod=date.today()
responses={'greetings':['Hello! How can I help you today?','Hi there! What can I do for you?','Hi! Nice to see you here!'],
          'name':['My name is HelpAI','I am HelpAI','I’m HelpAI, here to help you with any questions you have!','I’m HelpAI, nice to meet you! How can I help?'],
          'closing':["Goodbye! Have a great day ahead!",
"Take care! Looking forward to our next chat.",
"Bye for now! Don’t hesitate to come back if you have more questions.",
"See you soon! Wishing you all the best."],
          'date':[f'Today is {date_tod}',f'{date_tod}',f'Today\'s date is {date_tod}'],
          'wellness':['I\'m doing great, thank you for asking!',"I'm good, how about you?","I'm doing fine, thanks for checking in!"],
          'random':["Oh! Do you know Octopuses have three hearts—two pump blood to the gills, while the third pumps it to the rest of the body.",
                   "Ok. Here you go - The Eiffel Tower can grow taller in summer—the metal expands due to the heat, making the tower about 6 inches taller during hot weather.",
                   "Why don't skeletons fight each other?....... Because they don't have the guts!  HA HA HA"],
          'owner':["My Owner is Vysakh","I was programmed by Vysakh","My Owner's name is Vysakh","His name is Vysakh"]}

In [42]:
while True:
    print()
    k=input("User : ")
    res=preprocess(k)
    out_value=pred(res)
    res=np.argmax(out_value)
    thr=max(out_value[0])
    if thr >0.9:
        if res == 0:
            print(f"BOT : {random.choice(responses['greetings'])}")
        elif res == 1:
            print(f"BOT : {random.choice(responses['name'])}")
        elif res == 2:
            print(f"BOT : {random.choice(responses['date'])}")
        elif res ==3 :
            print(f"BOT : {random.choice(responses['closing'])}")
            break
        elif res ==4 :
            print(f"BOT : {random.choice(responses['wellness'])}")
        elif res ==5 :
            print(f"BOT : {random.choice(responses['random'])}")
        elif res ==6 :
            print(f"BOT : {random.choice(responses['owner'])}")
    else:
        print("Sorry.! I did'nt got that.Can you say it again..")
        
          



User : hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1.0
BOT : Hi! Nice to see you here!

User : how do you do
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
0.99999833
BOT : I'm doing great, thank you for asking!

User : what is today's date
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.999944
BOT : Today is 2024-11-16

User : ok say some fact
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.52172595
Sorry.! I did'nt got that.Can you say it again..

User : say something random 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1.0
BOT : Oh! Do you know Octopuses have three hearts—two pump blood to the gills, while the third pumps it to the rest of the body.

User : now what is your name
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
0.99542737
BOT : I’m HelpAI, here to help you with any questions you have!

User : your owner's name
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.99865675
BOT : My Owner is Vysakh

User : ok are you good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.66109353
Sorry.! I did'nt got that.Can you say it again..

User : a